In [8]:
import boto3
from pathlib import Path
from botocore.exceptions import ClientError

# ---- Config ----
bucket = "housing-regression-mlmodel"  # Change this to something unique
region = "eu-west-2"

PROJECT_ROOT = Path.cwd().parent
local_data_dir = PROJECT_ROOT / "data" / "processed"
local_model_dir = PROJECT_ROOT / "models"

s3 = boto3.client("s3", region_name=region)

# ---- Create bucket if it doesn't exist ----
try:
    s3.head_bucket(Bucket=bucket)
    print(f"✅ Bucket '{bucket}' already exists")
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == '404':
        print(f"📦 Creating bucket '{bucket}'...")
        if region == "us-east-1":
            s3.create_bucket(Bucket=bucket)
        else:
            s3.create_bucket(
                Bucket=bucket,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"✅ Bucket created")
    else:
        print(f"❌ Error: {e}")
        raise

# ---- Helper function ----
def upload_file(local_path: Path, s3_key: str):
    if not local_path.exists():
        print(f"❌ File not found: {local_path}")
        return
    print(f"⬆️ Uploading {local_path} → s3://{bucket}/{s3_key}")
    s3.upload_file(str(local_path), bucket, s3_key)

# ---- Upload files ----
upload_file(local_data_dir / "feature_engineered_holdout.csv", "processed/feature_engineered_holdout.csv")
upload_file(local_data_dir / "cleaning_holdout.csv", "processed/cleaning_holdout.csv")
upload_file(local_data_dir / "feature_engineered_train.csv", "processed/feature_engineered_train.csv")
upload_file(local_model_dir / "xgb_best_model.pkl", "models/xgb_best_model.pkl")

print("✅ Upload complete")

📦 Creating bucket 'housing-regression-mlmodel'...
✅ Bucket created
⬆️ Uploading /workspaces/Regression_ML_EndtoEnd/data/processed/feature_engineered_holdout.csv → s3://housing-regression-mlmodel/processed/feature_engineered_holdout.csv
⬆️ Uploading /workspaces/Regression_ML_EndtoEnd/data/processed/cleaning_holdout.csv → s3://housing-regression-mlmodel/processed/cleaning_holdout.csv
⬆️ Uploading /workspaces/Regression_ML_EndtoEnd/data/processed/feature_engineered_train.csv → s3://housing-regression-mlmodel/processed/feature_engineered_train.csv
⬆️ Uploading /workspaces/Regression_ML_EndtoEnd/models/xgb_best_model.pkl → s3://housing-regression-mlmodel/models/xgb_best_model.pkl
✅ Upload complete
